<a href="https://colab.research.google.com/github/jenkoj/nilm-gaf/blob/main/gaf_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Activation, MaxPooling2D, Input, Conv1D, GlobalAveragePooling1D

In [2]:
ls

sample_data/


In [4]:
#M - months A - appliances N - num of imgs 

#file_name ="UKDALE_GAF_1h_18M9A1000N"
#file_name= "UKDALE_GAF_1h_12M10A300N"
#file_name = "UKDALE_GAF_1h_12M10A500N"
file_name = "UKDALE_GAF_10m_100S0X_9A3000N" # x is n of paralel imgs


#check if we are in cloud or in PC
try:
    from google.colab import drive
    drive.mount('/content/drive')

    print("reading from cloud...")
    path = "/content/drive/MyDrive/Colab Notebooks/NILM/GAF_DS"
except:
    print("reading from HDD")
    path = "D:/jjenko/nilm data/GAF_DS"

#check if file exists
try:
    file = h5py.File(f"{path}""/"f"{file_name}"".hdf5","r+")
    print("file exists, reading it...")
except:
    print("error reading the file!")


# function to make reading easier
def read_many_hdf5(group_name,image_set_name):
    """ 
    Reads image from HDF5.
    """
    images = []

    # Open the HDF5 file
    file = h5py.File(f"{path}""/"f"{file_name}"".hdf5", "r+")

    images = np.array(file[f"{group_name}""/"f"{image_set_name}"])

    return images

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
reading from cloud...
file exists, reading it...


In [5]:
def basic_cnn_functional(X_train, num_classes, lr = 0.0001):
    
    input1 = Input(shape = X_train.shape[1: ])
    cnn = Conv2D(filters = 32, kernel_size = (7,7), strides = (2,2), activation='relu', padding = 'same')(input1)
    cnn1 = Conv2D(filters = 16, kernel_size =  (7,7), strides = (2,2), activation='relu', padding = 'same')(cnn)
    cnn2 = Conv2D(filters = 16, kernel_size =  (7,7), strides = (1,1), activation='relu', padding = 'same')(cnn1)
    cnn3 = Conv2D(filters = 8, kernel_size =  (7,7), strides = (1,1), activation='relu', padding = 'same')(cnn2)
    cnn4 = Conv2D(filters = 4, kernel_size =  (7,7), strides = (1,1), padding = 'same')(cnn3)
    act = Activation('relu')(cnn3)
    maxP = MaxPooling2D(pool_size = (2,2))(act)

    # prior layer should be flattend to be connected to dense layers
    Flatt = Flatten()(maxP)
    # dense layer with 50 neurons
    dense = Dense(32, activation = 'relu')(Flatt)
    # final layer with 10 neurons to classify the instances
    output = Dense(num_classes, activation = 'softmax')(dense)
    
    adam = optimizers.Adam(lr = lr)#lahko SGD uporabs tud
    model = keras.models.Model(inputs=input1, outputs=output)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model    


In [6]:
file.close()

In [7]:
def transfer_learning(X_train, num_classes, lr):
    base_model = keras.applications.ResNet50V2(
        weights=None,#'imagenet',  # Load weights pre-trained on ImageNet.
        input_shape=(300, 300, 1),
        include_top=False)  # Do not include the ImageNet classifier at the top.

    base_model.trainable = True

    inputs = keras.Input(shape = X_train.shape[1: ])
    # We make sure that the base_model is running in inference mode here,
    # by passing `training=False`. This is important for fine-tuning, as you will
    # learn in a few paragraphs.

    x = base_model(inputs, training=True)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
    model = keras.Model(inputs, outputs)

    model.compile(optimizer=keras.optimizers.Adam(lr = lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
#manualy_selected_appliances = ["computer monitor", "laptop computer", "washer dryer", "microwave","oven","boiler","toaster","kettle"]

#removed oven! - data looks fishy

manualy_selected_appliances = ["computer monitor", "laptop computer", "television", "washer dryer", "microwave","boiler","toaster","kettle","fridge"]

train_data = np.array(read_many_hdf5("data","gasf"))
label_data = np.array(read_many_hdf5("labels","gaf"))

print(label_data.shape)
print(train_data.shape)

(27000,)
(27000, 100, 100, 1)


In [11]:
num_frames = train_data.shape[0]
shape_x = train_data.shape[1]
shape_y = train_data.shape[2]
channel = train_data.shape[3]

num_of_classes = len(manualy_selected_appliances)

y_anomF = tf.keras.utils.to_categorical(label_data, num_classes=num_of_classes) #onehot encoding
X_resampled = train_data.reshape(train_data.shape[0],100, 100,1)
print(X_resampled.shape)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_anomF, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

#load model, add training data, n of classes and learning rate
model = basic_cnn_functional(X_train, num_of_classes, lr = 0.001)
#model = transfer_learning(X_train,num_of_classes,lr = 0.001)

class_weights = {0: 1.,
                1: 1.,
                2: 1.,
                3: 1.,
                4: 1.,
                5: 1.,
                6: 1.,
                7: 1.,
                8: 1.,
                9: 1,
                10: 1}

model.fit(X_train, y_train, batch_size = 16, validation_split = 0.2, epochs = 30, verbose = 1, class_weight=class_weights)

results = model.evaluate(X_test, y_test, verbose = 2)
print('Test accuracy: ', results[1])
Y_pred = model.predict(X_test, verbose = 2)
y_pred = np.argmax(Y_pred, axis=1)
Y_test = np.argmax(y_test, axis=-1)
print(confusion_matrix(Y_test, y_pred))
print(classification_report(Y_test, y_pred, target_names=manualy_selected_appliances))
#precision for various datassets - to see if more data helps
#window times for various appliances

(27000, 100, 100, 1)
(21600, 100, 100, 1)
(21600, 9)
Epoch 1/30
1080/1080 [==============================] - 36s 5ms/step - loss: 1.8763 - accuracy: 0.2922 - val_loss: 1.4322 - val_accuracy: 0.4692
Epoch 2/30
1080/1080 [==============================] - 5s 5ms/step - loss: 1.3960 - accuracy: 0.4796 - val_loss: 1.3037 - val_accuracy: 0.5213
Epoch 3/30
1080/1080 [==============================] - 5s 5ms/step - loss: 1.2489 - accuracy: 0.5329 - val_loss: 1.2996 - val_accuracy: 0.5264
Epoch 4/30
1080/1080 [==============================] - 5s 5ms/step - loss: 1.2207 - accuracy: 0.5495 - val_loss: 1.2582 - val_accuracy: 0.5484
Epoch 5/30
1080/1080 [==============================] - 5s 5ms/step - loss: 1.1339 - accuracy: 0.5699 - val_loss: 1.2388 - val_accuracy: 0.5546
Epoch 6/30
1080/1080 [==============================] - 5s 5ms/step - loss: 1.0864 - accuracy: 0.5959 - val_loss: 1.1951 - val_accuracy: 0.5711
Epoch 7/30
1080/1080 [==============================] - 5s 5ms/step - loss: 1.0250